<a href="https://colab.research.google.com/github/jserrataylor/cursoAI/blob/main/solicitud_dcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Cargar y procesar los datos
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Curso_AI/counseling_requests.csv")
data["timestamp"] = pd.to_datetime(data["timestamp"])

try:
    data["week_number"] = data["week"].str.extract("(\d+)").astype(int)
except KeyError:
    print("Error: No se encontró la columna 'week' en el conjunto de datos.")
    raise

data = data.set_index("timestamp")

# Preprocesamiento de los datos
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[["requests"]])

# Dividir los datos en conjuntos de entrenamiento y prueba
train_size = int(len(data_scaled) * 0.8)
train, test = data_scaled[0:train_size, :], data_scaled[train_size:len(data_scaled), :]

# Función para crear conjuntos de datos de series temporales
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        dataX.append(dataset[i:(i + look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Crear y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# Función para predecir la cantidad de estudiantes que solicitan consejería en una semana específica
def predict_counseling_requests(target_week):
    current_week = len(data_scaled) - look_back
    weeks_to_predict = target_week - current_week

    if weeks_to_predict < 0:
        weeks_to_predict += 52  # Si la semana ingresada ya pasó, asumimos que se refiere al próximo año.

    last_values = data_scaled[-look_back:]
    for _ in range(weeks_to_predict):
        last_values_expanded = np.expand_dims(last_values[-look_back:], axis=0)
        last_values_expanded = np.transpose(last_values_expanded, (0, 2, 1))  # Cambiar la forma de los datos de entrada
        prediction = model.predict(last_values_expanded)
        last_values = np.concatenate((last_values, prediction), axis=0)
    final_prediction = scaler.inverse_transform([last_values[-1]])[0, 0]
    return final_prediction


In [ ]:
input_week = int(input("Ingrese el número de la semana en el año para predecir la cantidad de estudiantes que solicitarán consejería (entre 1 y 52): "))
if 1 <= input_week <= 52:
    predicted_requests = predict_counseling_requests(input_week)
    print(f"Se espera que {predicted_requests:.0f} estudiantes soliciten servicios de consejería en la semana {input_week}.")
else:
    print("Por favor, ingrese un número de semana válido entre 1 y 52.")
